## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = range(1,1001)
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data,numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
# count
rdd.count()

1000

In [6]:
# first
rdd.first()

1

In [7]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [9]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [10]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([6.99635208e-01, 1.73212078e+00, 2.44754591e+00, 1.66529674e+00,
       2.13150856e+00, 4.98535373e+00, 1.72171589e-01, 2.32203152e+00,
       2.16001668e+00, 9.57047189e-02, 7.87149346e+00, 2.20374286e+00,
       1.01768559e+01, 5.32210976e+00, 6.52832748e+00, 2.39255201e+00,
       1.51415468e+01, 1.13929711e+01, 9.45608047e+00, 7.21879102e-01,
       3.51859818e-01, 5.92049725e+00, 1.29759379e+01, 1.63874800e+01,
       4.37049358e+00, 4.82315617e+00, 2.53274413e+01, 8.26572396e+00,
       2.33504178e+01, 2.21427965e+01, 1.81377461e+01, 2.34606385e+01,
       7.92345758e+00, 2.38304810e+01, 2.77717033e+01, 2.84356855e+01,
       1.47816681e+01, 1.99942718e+01, 3.67654224e+01, 3.54684527e+01,
       2.88990541e+01, 3.06126265e+01, 2.05411915e+01, 1.79822930e+01,
       3.03092636e+01, 1.80995676e+01, 4.07910713e+01, 9.08344976e+00,
       2.63306619e+01, 6.12692945e+00, 4.83146987e+01, 4.10832137e+01,
       2.83708608e+01, 2.90027428e-01, 2.65668841e+01, 9.12631218e+00,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [11]:
price_items = sc.parallelize(sales_figures, numSlices=10)

price_items.take(4)


[0.6996352075285022, 1.7321207780359238, 2.447545909585837, 1.6652967449967044]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [12]:
def sales_tax(num):
    return num * 1.08

revenue_plus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [13]:
# perform action to retrieve rdd values

revenue_plus_tax.take(10)

[0.7556060241307824,
 1.8706904402787978,
 2.6433495823527045,
 1.7985204845964409,
 2.302029241242236,
 5.384182033495759,
 0.1859453162261055,
 2.5077940434484556,
 2.3328180136597165,
 0.10336109636929064]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [14]:
discounted = price_items.map(lambda x: x*0.9)

In [15]:
discounted.take(10)

[0.629671686775652,
 1.5589087002323314,
 2.2027913186272534,
 1.498767070497034,
 1.9183577010351966,
 4.486818361246466,
 0.15495443018842123,
 2.0898283695403794,
 1.9440150113830972,
 0.08613424697440886]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [16]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)


[928.6758239838194,
 913.6812533078643,
 908.7429837057277,
 906.6937278958961,
 898.2756972825699,
 893.8414874458149,
 890.9713397548236,
 890.8618345408172,
 885.946359766842,
 885.0289122877525,
 883.5383933607008,
 869.4161629437428,
 868.3651624305479,
 866.8820630876909,
 863.3210712892887]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [17]:
price_items.toDebugString()

b'(10) ParallelCollectionRDD[6] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [18]:
mapped = price_items.map(lambda x: (x, x*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.6996352075285022, 0.629671686775652), (1.7321207780359238, 1.5589087002323314), (2.447545909585837, 2.2027913186272534), (1.6652967449967044, 1.498767070497034), (2.131508556705774, 1.9183577010351966), (4.985353734718295, 4.486818361246466), (0.17217158909824581, 0.15495443018842123), (2.3220315217115326, 2.0898283695403794), (2.1600166793145523, 1.9440150113830972), (0.09570471886045429, 0.08613424697440886)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [19]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.6996352075285022, 0.629671686775652, 1.7321207780359238, 1.5589087002323314, 2.447545909585837, 2.2027913186272534, 1.6652967449967044, 1.498767070497034, 2.131508556705774, 1.9183577010351966]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [20]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)
selected_items.count()
# calculate total remaining in inventory 


311

In [21]:
selected_items.take(10)

[317.12007546258366,
 301.8008280496141,
 340.50225649776405,
 358.2231205558331,
 359.6385632320573,
 337.70901385878585,
 321.4346079066274,
 315.84487360059086,
 355.5665211348986,
 349.7887749229385]

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [22]:
selected_items.reduce(lambda x,y :x + y)


148145.51345746042

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [23]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(31, 317.12007546258366),
 (40, 301.8008280496141),
 (15, 340.50225649776405),
 (15, 358.2231205558331),
 (3, 359.6385632320573),
 (29, 337.70901385878585),
 (31, 321.4346079066274)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [24]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y :x + y)
total_spent.take(10)

[(30, 3556.841743238362),
 (50, 1977.9530309267616),
 (10, 3377.535559091321),
 (40, 2374.2293093075064),
 (20, 1435.1110699618164),
 (1, 4508.578115304324),
 (21, 2102.2171193795075),
 (31, 3335.1636864993748),
 (11, 2784.9443426657417),
 (41, 2471.258666494031)]

In [25]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(33, 5708.688761074429),
 (29, 5304.309978991503),
 (14, 4701.83007055275),
 (47, 4537.610643362829),
 (1, 4508.578115304324),
 (39, 4452.977940203902),
 (7, 4427.061359155316),
 (3, 4422.615394887337),
 (25, 4416.496391348866),
 (8, 4405.47696991767),
 (15, 4063.398253292757),
 (5, 3888.8317370904692),
 (46, 3875.8928843983163),
 (23, 3872.374672365789),
 (28, 3567.7133293140996),
 (30, 3556.841743238362),
 (10, 3377.535559091321),
 (31, 3335.1636864993748),
 (37, 3326.8498782118113),
 (12, 3305.9066108295656),
 (36, 3229.0018084299445),
 (24, 3203.482599863446),
 (26, 3135.8357770650096),
 (48, 3035.107083763628),
 (38, 2891.0682635032063),
 (6, 2822.6866247808375),
 (22, 2805.0944123585505),
 (11, 2784.9443426657417),
 (27, 2733.1349481380357),
 (43, 2727.9512343109955),
 (18, 2635.605111446109),
 (34, 2631.6578869677414),
 (41, 2471.258666494031),
 (40, 2374.2293093075064),
 (4, 2359.2691885396507),
 (13, 2336.764159562851),
 (21, 2102.2171193795075),
 (50, 1977.9530309267616),
 (

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [26]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(3, 12),
 (22, 11),
 (33, 11),
 (39, 10),
 (29, 10),
 (46, 10),
 (26, 10),
 (11, 10),
 (34, 9),
 (32, 9),
 (12, 9),
 (50, 8),
 (5, 8),
 (18, 8),
 (16, 7),
 (24, 7),
 (38, 7),
 (28, 7),
 (8, 7),
 (6, 7),
 (37, 6),
 (25, 6),
 (15, 6),
 (2, 6),
 (40, 6),
 (14, 5),
 (23, 5),
 (36, 5),
 (47, 5),
 (17, 5),
 (4, 5),
 (31, 5),
 (49, 5),
 (19, 5),
 (35, 5),
 (42, 5),
 (45, 5),
 (41, 5),
 (7, 4),
 (9, 4),
 (20, 4),
 (48, 4),
 (44, 4),
 (21, 4),
 (43, 3),
 (27, 3),
 (30, 3),
 (10, 2),
 (1, 2),
 (13, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
